In [22]:
import pickle
import mlflow
import uuid

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [4]:
year = 2024
month = 1
taxi_type = 'green'

input_file = f'./data/{taxi_type}_tripdata_{year:04d}-{month:02d}.csv'
output_file = f'./output/{taxi_type}_tripdata_{year:04d}-{month:02d}.csv'

In [5]:
from sklearn.pipeline import make_pipeline
from mlflow.tracking import MlflowClient
import xgboost as xgb

In [6]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [8]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1741341672713, experiment_id='2', last_update_time=1741341672713, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [ ]:
RUN_ID = 'e73c0f6b3b754e80bcfb3be72c0c2f40'

logged_model = f'runs:/{RUN_ID}/model'
model = mlflow.pyfunc.load_model(logged_model)

In [10]:
# dv = client.download_artifacts(run_id=RUN_ID, path='preprocessor/preprocessor.b', dst_path='./models')

with open("models/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [11]:
dv

DictVectorizer()

In [12]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: e73c0f6b3b754e80bcfb3be72c0c2f40

In [ ]:
def read_dataframe(filename: str):
    df = pd.read_csv(filename)

    df['duration'] = pd.to_datetime(df["lpep_dropoff_datetime"]) - pd.to_datetime(df["lpep_pickup_datetime"])
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df['ride_ids'] = generate_uuids(len(df))
    return df


def prepare_data(df: pd.DataFrame, dv: DictVectorizer):
    duration = df["duration"].to_numpy()
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    X = dv.transform(dicts)
    # X = xgb.DMatrix(X, label=duration)
    return X

In [ ]:
def load_model(run_id):
    logged_model = f'runs:/{run_id}/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model

def apply_model(input_file, dv, run_id, output_file):
    df = read_dataframe(input_file)
    model = load_model(run_id)

    X = prepare_data(df, dv)
    y_pred = model.predict(X)

    df_result = pd.DataFrame()
    df_result['ride_ids'] = df['ride_ids']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id

    df_result.to_csv(output_file, index=False)

### Generate result dataframe

In [16]:
str(uuid.uuid4())

'5f451f57-5bf3-4bb2-899c-e294890a9a9d'

In [25]:
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

In [23]:
df_result = pd.DataFrame()

In [26]:
df['ride_ids'] = generate_uuids(len(df))

In [27]:
df_result['ride_ids'] = df['ride_ids']
df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
df_result['PULocationID'] = df['PULocationID']
df_result['DOLocationID'] = df['DOLocationID']
df_result['actual_duration'] = df['duration']
df_result['predicted_duration'] = y_pred
df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
df_result['model_version'] = RUN_ID
# df_result['lpep_dropoff_datetime'] = df['lpep_dropoff_datetime']

In [21]:
df_result.to_csv(output_file, index=False)

In [ ]:
apply_model(input_file, RUN_ID, output_file)